# Google Merchandise Store MTA Data Wrangling

We need to convert the raw clickstream to something we can do sequential MTA on. Also some intresting fields are nested so this data is techinically semi-structured and it might be good to flatten. 

The final result of this notebook includes:

 - a sequence fact table, which has both conversion and dead end sequences all assigned a sequence id
 - a visitor id detail table, which has 1 visitor id and some intresting details about the visitor such as browser, os, isp provider, ip address geography etc.

In [ ]:
import pandas as pd
import numpy as np
import json
import secrets

pd.options.display.max_columns = None

In [2]:
folder = '../datasets/' # the ../ represents moving one folder back from current directory 
df = pd.read_csv(folder + 'clickstream_raw.csv')

In [3]:
df.head()

,channelGrouping,customDimensions,date,device,fullVisitorId,geoNetwork,hits,socialEngagementType,totals,trafficSource,visitId,visitNumber,visitStartTime
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",1259490915281096752,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '7', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508165159,2,1508165159
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171016,"{""browser"": ""Safari"", ""browserVersion"": ""not a...",5983987899933814948,"{""continent"": ""Americas"", ""subContinent"": ""Nor...","[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508177658,1,1508177658
2,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",0875141984250977091,"{""continent"": ""Asia"", ""subContinent"": ""Eastern...","[{'hitNumber': '1', 'time': '0', 'hour': '23',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508223135,1,1508223135
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",451521411412093630,"{""continent"": ""Europe"", ""subContinent"": ""Weste...","[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""2"", ""pageviews"": ""2"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508160016,3,1508160016
4,Organic Search,[],20171016,"{""browser"": ""Chrome"", ""browserVersion"": ""not a...",6723102816157399269,"{""continent"": ""Europe"", ""subContinent"": ""Easte...","[{'hitNumber': '1', 'time': '0', 'hour': '12',...",Not Socially Engaged,"{""visits"": ""1"", ""hits"": ""3"", ""pageviews"": ""3"",...","{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508180825,1,1508180825


In [4]:
def flatten_json(df, column_names_list):
    """
    This function will take a df and a list of column names to flatten
    """
    for col in column_names_list:
        # Convert the JSON string to a dictionary
        df[col] = df[col].apply(lambda x: json.loads(x))
        json_dict = pd.json_normalize(df[col])

        # Create new columns for each key-value pairing in the dictionary
        prefix = col + '_'
        for key in json_dict.columns:
            new_col_name = prefix + key
            df[new_col_name] = json_dict[key]

        # Drop the original column that contained the JSON string
        df = df.drop(columns=[col])

    return df

In [5]:
df2 = flatten_json(df,['device','geoNetwork','totals'])
df2.head()

,channelGrouping,customDimensions,date,fullVisitorId,hits,socialEngagementType,trafficSource,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,device_browserSize,device_operatingSystem,device_operatingSystemVersion,device_isMobile,device_mobileDeviceBranding,device_mobileDeviceModel,device_mobileInputSelector,device_mobileDeviceInfo,device_mobileDeviceMarketingName,device_flashVersion,device_language,device_screenColors,device_screenResolution,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_region,geoNetwork_metro,geoNetwork_city,geoNetwork_cityId,geoNetwork_networkDomain,geoNetwork_latitude,geoNetwork_longitude,geoNetwork_networkLocation,totals_visits,totals_hits,totals_pageviews,totals_timeOnSite,totals_sessionQualityDim,totals_newVisits,totals_transactions,totals_transactionRevenue,totals_totalTransactionRevenue,totals_bounces
0,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,1259490915281096752,"[{'hitNumber': '1', 'time': '0', 'hour': '7', ...",Not Socially Engaged,"{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508165159,2,1508165159,Safari,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Europe,Western Europe,Netherlands,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,32,1,NaN,NaN,NaN,NaN,NaN
1,Organic Search,"[{'index': '4', 'value': 'North America'}]",20171016,5983987899933814948,"[{'hitNumber': '1', 'time': '0', 'hour': '11',...",Not Socially Engaged,"{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508177658,1,1508177658,Safari,not available in demo dataset,not available in demo dataset,iOS,not available in demo dataset,True,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,mobile,Americas,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,bhn.net,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,51,1,1,NaN,NaN,NaN,NaN
2,Organic Search,"[{'index': '4', 'value': 'APAC'}]",20171016,0875141984250977091,"[{'hitNumber': '1', 'time': '0', 'hour': '23',...",Not Socially Engaged,"{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508223135,1,1508223135,Chrome,not available in demo dataset,not available in demo dataset,Linux,not available in demo dataset,False,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,desktop,Asia,Eastern Asia,Taiwan,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,nthu.edu.tw,not available in demo dataset,not available in demo dataset,not available in demo dataset,1,2,2,31,1,1,NaN,NaN,NaN,NaN
3,Organic Search,"[{'index': '4', 'value': 'EMEA'}]",20171016,451521411412093630,"[{'hitNumber': '1', 'time': '0', 'hour': '6', ...",Not Socially Engaged,"{""campaign"": ""(not set)"", ""source"": ""google"", ...",1508160016,3,1508160016,Chrome,not available in demo dataset,not available in demo dataset,Windows,not available in demo dataset,False,not availab

In [37]:
df2.fillna(0,inplace=True)

df2['visitStart_datetime'] = pd.to_datetime(df2['visitStartTime'], unit='s')

df2['visitEnd_datetime'] = pd.to_datetime(pd.to_numeric(df2['visitStartTime'])+pd.to_numeric(df2['totals_timeOnSite']), unit='s')

# Add one second to Visit End Datetime if equal to Visit Start Datetime
df2.loc[df2['visitStart_datetime'] == df2['visitEnd_datetime'], 'visitEnd_datetime'] += np.timedelta64(1, 's')


## hardcode of relevant columns

cols_to_keep = ['channelGrouping','visitStart_datetime','visitEnd_datetime'
            ,'visitNumber','fullVisitorId','device_browser'
            ,'device_operatingSystem','device_isMobile','device_deviceCategory'
            ,'geoNetwork_continent','geoNetwork_subContinent','geoNetwork_country'
            ,'geoNetwork_metro','geoNetwork_networkDomain','totals_pageviews'
            ,'totals_timeOnSite', 'totals_newVisits','totals_transactions'
            ,'totals_bounces']

df3 = df2[cols_to_keep]

num_cols = ['visitNumber','totals_pageviews','totals_timeOnSite'
            ,'totals_newVisits','totals_transactions','totals_bounces']


def convert_columns_to_numeric(df, columns):
    """
    This function takes any list of column names and will convert those to numeric
    """
    
    df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')
    return df

df3 = convert_columns_to_numeric(df3,num_cols)

df3.sort_values(by=['fullVisitorId','visitStart_datetime'],inplace=True)

df3.head()

C:\Users\TEMPLSD\AppData\Local\Temp\ipykernel_22388\3165799986.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')
C:\Users\TEMPLSD\AppData\Local\Temp\ipykernel_22388\3165799986.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.sort_values(by=['fullVisitorId','visitStart_datetime'],inplace=True)


,channelGrouping,visitStart_datetime,visitEnd_datetime,visitNumber,fullVisitorId,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,totals_pageviews,totals_timeOnSite,totals_newVisits,totals_transactions,totals_bounces
43681,Organic Search,2017-06-01 18:13:16,2017-06-01 18:16:47,1,0000650183049109374,Chrome,Android,True,mobile,Asia,Southern Asia,India,(not set),(not set),10,211,1,0,0
1145,Organic Search,2017-02-03 21:35:41,2017-02-03 21:35:42,1,0000750929315523353,Safari,Macintosh,False,desktop,Americas,Northern America,United States,not available in demo dataset,zoominternet.net,1,0,1,0,1
45907,Display,2017-09-17 01:24:48,2017-09-17 01:24:49,1,000094154452308463,Safari,iOS,True,tablet,Americas,Northern America,United States,not available in demo dataset,verizon.net,1,0,1,0,1
45939,Display,2017-09-17 02:01:20,2017-09-17 02:01:21,2,000094154452308463,Safari,iOS,True,tablet,Americas,Northern America,United States,not available in demo dataset,verizon.net,1,0,0,0,1
36781,Direct,2017-10-12 01:02:05,2017-10-12 01:03:43,1,000106364568517340,Chrome,Windows,False,desktop,Americas,South America,Peru,(not set),uns.edu.pe,2,98,1,0,0


In [38]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 121782 entries, 43681 to 108146
Data columns (total 19 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   channelGrouping           121782 non-null  object        
 1   visitStart_datetime       121782 non-null  datetime64[ns]
 2   visitEnd_datetime         121782 non-null  datetime64[ns]
 3   visitNumber               121782 non-null  int64         
 4   fullVisitorId             121782 non-null  object        
 5   device_browser            121782 non-null  object        
 6   device_operatingSystem    121782 non-null  object        
 7   device_isMobile           121782 non-null  bool          
 8   device_deviceCategory     121782 non-null  object        
 9   geoNetwork_continent      121782 non-null  object        
 10  geoNetwork_subContinent   121782 non-null  object        
 11  geoNetwork_country        121782 non-null  object        
 12

In [39]:
## An example of a single visitor that has four website visits 0 conversions
df3.loc[df3['fullVisitorId']=='451521411412093630',:].sort_values(by=['visitStart_datetime'])

,channelGrouping,visitStart_datetime,visitEnd_datetime,visitNumber,fullVisitorId,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,totals_pageviews,totals_timeOnSite,totals_newVisits,totals_transactions,totals_bounces
16306,Direct,2017-10-12 18:27:02,2017-10-12 18:41:04,1,451521411412093630,Chrome,Windows,False,desktop,Europe,Western Europe,Belgium,not available in demo dataset,(not set),5,842,1,0,0
16161,Organic Search,2017-10-12 18:56:44,2017-10-12 19:15:14,2,451521411412093630,Chrome,Windows,False,desktop,Europe,Western Europe,Belgium,not available in demo dataset,(not set),3,1110,0,0,0
3,Organic Search,2017-10-16 13:20:16,2017-10-16 13:29:47,3,451521411412093630,Chrome,Windows,False,desktop,Europe,Western Europe,Belgium,(not set),(not set),2,571,0,0,0
164,Organic Search,2017-10-16 14:06:30,2017-10-16 14:06:31,4,451521411412093630,Chrome,Windows,False,desktop,Europe,Western Europe,Belgium,(not set),(not set),1,0,0,0,1


## Assumptions to make visitor control variables table

<b> 1 We will take the visitor characteristics from the very first visit just so we guarantee to get a 1 to 1 mapping of id to characteristics </b>


In [40]:
# This gives us a first row for every unique visitor id to filter on. 
# For some reason cannot rely on newvisits in rare cases a single visitor id can have more than one newvisit

df3['obs_count'] = df3.groupby('fullVisitorId').cumcount() + 1

detail_cols = df3.columns[5:13]


visitor_detail_df = df3.loc[df3['obs_count']==1, ['fullVisitorId'] + list(detail_cols)].reset_index(drop=True)
visitor_detail_df.head()

C:\Users\TEMPLSD\AppData\Local\Temp\ipykernel_22388\3380343767.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['obs_count'] = df3.groupby('fullVisitorId').cumcount() + 1


,fullVisitorId,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_metro
0,0000650183049109374,Chrome,Android,True,mobile,Asia,Southern Asia,India,(not set)
1,0000750929315523353,Safari,Macintosh,False,desktop,Americas,Northern America,United States,not available in demo dataset
2,000094154452308463,Safari,iOS,True,tablet,Americas,Northern America,United States,not available in demo dataset
3,000106364568517340,Chrome,Windows,False,desktop,Americas,South America,Peru,(not set)
4,0001348344294054462,Chrome,Linux,False,desktop,Americas,Northern America,United States,New York NY


In [41]:
## this df has one visitor id and then some potential control variables we associate to that visitor id
visitor_detail_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99613 entries, 0 to 99612
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   fullVisitorId            99613 non-null  object
 1   device_browser           99613 non-null  object
 2   device_operatingSystem   99613 non-null  object
 3   device_isMobile          99613 non-null  bool  
 4   device_deviceCategory    99613 non-null  object
 5   geoNetwork_continent     99613 non-null  object
 6   geoNetwork_subContinent  99613 non-null  object
 7   geoNetwork_country       99613 non-null  object
 8   geoNetwork_metro         99613 non-null  object
dtypes: bool(1), object(8)
memory usage: 6.2+ MB


In [42]:
visitor_detail_df['fullVisitorId'].nunique()

99613

## Assumptions to make sequencing data

This is always a very tricky step of multi touch attribution. Think carefully about the business problem and what the "population" ought to be. Think *very* carefully about the negative cases or examples you want machine learning to consider.

E.g. If you are doing a 90 day lookback window on conversions to formulate positive cases to model on how would you formulate the 90 day window on customer journies that result in no conversion? See #3 postive case and negative case rationale for the assumption for this motivating example.

<b> 1 When a website visit contains a conversion, the vist end datetime is the conversion timestamp </b>

<b> 2 When a website visit starts the channelgrouping and the visit start datetime form the touchpoint timestamp </b>

<b> 3 Positve Case and Negative Case Rationale  </b>

- positive cases: any time there is a conversion look back at all the channels that full visitor id used prior
- negative cases: any time there is a visitor id with no conversions lets call the very last visit end datetime the psuedo negative conversion and pull all channels used prior

<b> 4 If there were sequence activities before or after the dataset timeframe these were truncated </b>

#### grab positive sequences

a single visitor id could have many vistis with a transaction, so a single visitor id could generate multiple customer journies to model on

In [43]:
conversion_df = df3.loc[df3['totals_transactions'] >= 1,['fullVisitorId','visitEnd_datetime']]
conversion_df['event_name'] = 'conversion'
conversion_df['event_datetime'] = conversion_df['visitEnd_datetime']
conversion_df['conversion_proximity'] = 0
conversion_df = conversion_df[['fullVisitorId','event_name','event_datetime','conversion_proximity']]
conversion_df.reset_index(drop=True,inplace=True)
conversion_df.head()

,fullVisitorId,event_name,event_datetime,conversion_proximity
0,0003961110741104601,conversion,2017-05-22 04:09:44,0
1,0084383590795093998,conversion,2016-09-07 22:03:52,0
2,0086819414174186743,conversion,2018-02-05 15:41:14,0
3,0092577710025173656,conversion,2016-10-18 00:10:57,0
4,0121011502564299055,conversion,2018-02-09 13:20:40,0


#### conversion path ids

We will need to keep track of conversion paths for modeling so we will generate our own unique conversion path ids

In [44]:
def generate_unique_ids(length):
    alphabet = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789"
    return ''.join(secrets.choice(alphabet) for _ in range(length))

customer_ids_1 = []
while len(customer_ids_1) < len(conversion_df):
    customer_ids_1.append(generate_unique_ids(15))
    
conversion_df['sequence_id'] = customer_ids_1
conversion_df.head()

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
0,0003961110741104601,conversion,2017-05-22 04:09:44,0,dU5UOwplcDCWn7Z
1,0084383590795093998,conversion,2016-09-07 22:03:52,0,bXwLU1c6w7oGnm8
2,0086819414174186743,conversion,2018-02-05 15:41:14,0,VLB22UcgcwWbqYi
3,0092577710025173656,conversion,2016-10-18 00:10:57,0,pkzACIzTGu73XaJ
4,0121011502564299055,conversion,2018-02-09 13:20:40,0,2kUVU2Hsi6R8UDO


In [45]:
conversion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   fullVisitorId         1337 non-null   object        
 1   event_name            1337 non-null   object        
 2   event_datetime        1337 non-null   datetime64[ns]
 3   conversion_proximity  1337 non-null   int64         
 4   sequence_id           1337 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 52.4+ KB


In [46]:
df3.head()

,channelGrouping,visitStart_datetime,visitEnd_datetime,visitNumber,fullVisitorId,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,totals_pageviews,totals_timeOnSite,totals_newVisits,totals_transactions,totals_bounces,obs_count
43681,Organic Search,2017-06-01 18:13:16,2017-06-01 18:16:47,1,0000650183049109374,Chrome,Android,True,mobile,Asia,Southern Asia,India,(not set),(not set),10,211,1,0,0,1
1145,Organic Search,2017-02-03 21:35:41,2017-02-03 21:35:42,1,0000750929315523353,Safari,Macintosh,False,desktop,Americas,Northern America,United States,not available in demo dataset,zoominternet.net,1,0,1,0,1,1
45907,Display,2017-09-17 01:24:48,2017-09-17 01:24:49,1,000094154452308463,Safari,iOS,True,tablet,Americas,Northern America,United States,not available in demo dataset,verizon.net,1,0,1,0,1,1
45939,Display,2017-09-17 02:01:20,2017-09-17 02:01:21,2,000094154452308463,Safari,iOS,True,tablet,Americas,Northern America,United States,not available in demo dataset,verizon.net,1,0,0,0,1,2
36781,Direct,2017-10-12 01:02:05,2017-10-12 01:03:43,1,000106364568517340,Chrome,Windows,False,desktop,Americas,South America,Peru,(not set),uns.edu.pe,2,98,1,0,0,1


In [47]:
## start with channel detail table, left join conversion details, calculate the istance in seconds between each visit channel start and 

df4 = df3.merge(conversion_df,how='left',on='fullVisitorId')
df4 = df4.dropna(subset=['event_name'])
df4['conversion_proximity'] = (df4['event_datetime'] - df4['visitStart_datetime']).dt.total_seconds()
df4 = df4.loc[df4['conversion_proximity']>=0,:]
df4 = df4[['fullVisitorId','channelGrouping','visitStart_datetime','conversion_proximity','sequence_id']]
df4.columns = ['fullVisitorId','event_name','event_datetime','conversion_proximity','sequence_id']
df4.head()

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
15,0003961110741104601,Organic Search,2017-05-22 04:01:22,502.0,dU5UOwplcDCWn7Z
378,0084383590795093998,Referral,2016-09-07 21:58:00,352.0,bXwLU1c6w7oGnm8
387,0086819414174186743,Organic Search,2018-02-05 15:25:08,966.0,VLB22UcgcwWbqYi
419,0092577710025173656,Organic Search,2016-10-17 20:57:11,11626.0,pkzACIzTGu73XaJ
420,0092577710025173656,Organic Search,2016-10-17 23:00:58,4199.0,pkzACIzTGu73XaJ


In [48]:
transactions_df = pd.concat([df4,conversion_df],ignore_index=True)
transactions_df.sort_values(by=['fullVisitorId','conversion_proximity'], ascending=[True,False], inplace=True)
transactions_df['event_name'] = transactions_df['event_name'].str.lower().str.replace(' ','_')
transactions_df.reset_index(drop=True,inplace=True)
transactions_df.head(15)

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
0,0003961110741104601,organic_search,2017-05-22 04:01:22,502.0,dU5UOwplcDCWn7Z
1,0003961110741104601,conversion,2017-05-22 04:09:44,0.0,dU5UOwplcDCWn7Z
2,0084383590795093998,referral,2016-09-07 21:58:00,352.0,bXwLU1c6w7oGnm8
3,0084383590795093998,conversion,2016-09-07 22:03:52,0.0,bXwLU1c6w7oGnm8
4,0086819414174186743,organic_search,2018-02-05 15:25:08,966.0,VLB22UcgcwWbqYi
5,0086819414174186743,conversion,2018-02-05 15:41:14,0.0,VLB22UcgcwWbqYi
6,0092577710025173656,organic_search,2016-10-17 20:57:11,11626.0,pkzACIzTGu73XaJ
7,0092577710025173656,organic_search,2016-10-17 23:00:58,4199.0,pkzACIzTGu73XaJ
8,0092577710025173656,conversion,2016-10-18 00:10:57,0.0,pkzACIzTGu73XaJ
9,0121011502564299055,direct,2017-11-08 17:00:09,8629475.0,QNZPZitPL4zAB1t


In [49]:
## here is an example of a visitorid with 6 conversions, note the first referral 01/10/2017 gets repeated 6 times
id_counts = transactions_df.loc[transactions_df['event_name']=='conversion']['fullVisitorId'].value_counts()
id_counts_2p = id_counts[id_counts >= 2].index.tolist()
transactions_df.loc[transactions_df['fullVisitorId']==id_counts_2p[0],:].sort_values(by=['event_datetime'])

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
126,0608915197735218105,referral,2017-01-10 18:03:51,6916834.0,Y4oyYAHXnur2kSl
128,0608915197735218105,referral,2017-01-10 18:03:51,5702690.0,9EISjNWDQrTDLtz
149,0608915197735218105,referral,2017-01-10 18:03:51,647.0,gvzdWs8NHkWhGqR
143,0608915197735218105,referral,2017-01-10 18:03:51,276615.0,OpI4gRy5JACBAdm
132,0608915197735218105,referral,2017-01-10 18:03:51,5364808.0,hDlvSlzb4DcKWYU
139,0608915197735218105,referral,2017-01-10 18:03:51,1396866.0,I7Rnwz88SG6wVRC
151,0608915197735218105,conversion,2017-01-10 18:14:38,0.0,gvzdWs8NHkWhGqR
146,0608915197735218105,referral,2017-01-13 22:26:32,1654.0,OpI4gRy5JACBAdm
141,0608915197735218105,referral,2017-01-13 22:26:32,1121905.0,I7Rnwz88SG6wVRC
133,0608915197735218105,referral,2017-01-13 22:26:32,5089847.0,hDlvSlzb4DcKWYU


In [50]:
## if you sort by converson path id fist you will better see the sequences that will be generated by the data to study

transactions_df.loc[transactions_df['fullVisitorId']==id_counts_2p[0],:].sort_values(by=['sequence_id','event_datetime'])

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
128,0608915197735218105,referral,2017-01-10 18:03:51,5702690.0,9EISjNWDQrTDLtz
131,0608915197735218105,referral,2017-01-13 22:26:32,5427729.0,9EISjNWDQrTDLtz
134,0608915197735218105,referral,2017-01-26 18:58:41,4317000.0,9EISjNWDQrTDLtz
135,0608915197735218105,referral,2017-01-26 21:54:18,4306463.0,9EISjNWDQrTDLtz
142,0608915197735218105,referral,2017-03-13 19:26:29,340932.0,9EISjNWDQrTDLtz
147,0608915197735218105,referral,2017-03-17 17:54:33,848.0,9EISjNWDQrTDLtz
155,0608915197735218105,conversion,2017-03-17 18:08:41,0.0,9EISjNWDQrTDLtz
139,0608915197735218105,referral,2017-01-10 18:03:51,1396866.0,I7Rnwz88SG6wVRC
141,0608915197735218105,referral,2017-01-13 22:26:32,1121905.0,I7Rnwz88SG6wVRC
144,0608915197735218105,referral,2017-01-26 18:58:41,11176.0,I7Rnwz88SG6wVRC


#### grab negative sequences

We will grab all visitor ids with 0 transactions.

We will query their very last visit end datetime as a dead end event. 

Then just like the conversion sequences we will assign a dead end a sequence id

In [51]:
df3.head()

,channelGrouping,visitStart_datetime,visitEnd_datetime,visitNumber,fullVisitorId,device_browser,device_operatingSystem,device_isMobile,device_deviceCategory,geoNetwork_continent,geoNetwork_subContinent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,totals_pageviews,totals_timeOnSite,totals_newVisits,totals_transactions,totals_bounces,obs_count
43681,Organic Search,2017-06-01 18:13:16,2017-06-01 18:16:47,1,0000650183049109374,Chrome,Android,True,mobile,Asia,Southern Asia,India,(not set),(not set),10,211,1,0,0,1
1145,Organic Search,2017-02-03 21:35:41,2017-02-03 21:35:42,1,0000750929315523353,Safari,Macintosh,False,desktop,Americas,Northern America,United States,not available in demo dataset,zoominternet.net,1,0,1,0,1,1
45907,Display,2017-09-17 01:24:48,2017-09-17 01:24:49,1,000094154452308463,Safari,iOS,True,tablet,Americas,Northern America,United States,not available in demo dataset,verizon.net,1,0,1,0,1,1
45939,Display,2017-09-17 02:01:20,2017-09-17 02:01:21,2,000094154452308463,Safari,iOS,True,tablet,Americas,Northern America,United States,not available in demo dataset,verizon.net,1,0,0,0,1,2
36781,Direct,2017-10-12 01:02:05,2017-10-12 01:03:43,1,000106364568517340,Chrome,Windows,False,desktop,Americas,South America,Peru,(not set),uns.edu.pe,2,98,1,0,0,1


In [52]:
dead_ends_df = df3.groupby(['fullVisitorId'],as_index=False).agg(
    total_transactions = ('totals_transactions',np.sum)
    ,max_visitend = ('visitEnd_datetime',np.max)
)
dead_ends_df = dead_ends_df.loc[dead_ends_df['total_transactions']==0,:]
dead_ends_df['event_name'] = 'dead_end'
dead_ends_df['conversion_proximity'] = 0

dead_ends_df = dead_ends_df[['fullVisitorId','event_name','max_visitend','conversion_proximity']]
dead_ends_df.columns = ['fullVisitorId','event_name','event_datetime', 'conversion_proximity']

customer_ids_2 = []
while len(customer_ids_2) < len(dead_ends_df):
    customer_ids_2.append(generate_unique_ids(15))
    
dead_ends_df['sequence_id'] = customer_ids_2
dead_ends_df.head()

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
0,0000650183049109374,dead_end,2017-06-01 18:16:47,0,nW31ahtlgEpFWi1
1,0000750929315523353,dead_end,2017-02-03 21:35:42,0,f6X79WOxA3jUxPU
2,000094154452308463,dead_end,2017-09-17 02:01:21,0,L5wAoLtd1C6Cvvc
3,000106364568517340,dead_end,2017-10-12 01:03:43,0,maGY5aACPGJoF9o
4,0001348344294054462,dead_end,2017-06-05 15:06:25,0,VCl4OtCpaCafWUv


In [53]:
df5 = df3.merge(dead_ends_df,how='left',on='fullVisitorId')
df5 = df5.dropna(subset=['event_name'])
df5['conversion_proximity'] = (df5['event_datetime'] - df5['visitStart_datetime']).dt.total_seconds()
df5 = df5.loc[df5['conversion_proximity']>=0,:]
df5 = df5[['fullVisitorId','channelGrouping','visitStart_datetime','conversion_proximity','sequence_id']]
df5.columns = ['fullVisitorId','event_name','event_datetime','conversion_proximity','sequence_id']
df5.head()

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
0,0000650183049109374,Organic Search,2017-06-01 18:13:16,211.0,nW31ahtlgEpFWi1
1,0000750929315523353,Organic Search,2017-02-03 21:35:41,1.0,f6X79WOxA3jUxPU
2,000094154452308463,Display,2017-09-17 01:24:48,2193.0,L5wAoLtd1C6Cvvc
3,000094154452308463,Display,2017-09-17 02:01:20,1.0,L5wAoLtd1C6Cvvc
4,000106364568517340,Direct,2017-10-12 01:02:05,98.0,maGY5aACPGJoF9o


In [54]:
transactions_df2 = pd.concat([df5,dead_ends_df],ignore_index=True)
transactions_df2.sort_values(by=['fullVisitorId','conversion_proximity'], ascending=[True,False], inplace=True)
transactions_df2['event_name'] = transactions_df2['event_name'].str.lower().str.replace(' ','_')
transactions_df2.reset_index(drop=True,inplace=True)
transactions_df2.head(15)

,fullVisitorId,event_name,event_datetime,conversion_proximity,sequence_id
0,0000650183049109374,organic_search,2017-06-01 18:13:16,211.0,nW31ahtlgEpFWi1
1,0000650183049109374,dead_end,2017-06-01 18:16:47,0.0,nW31ahtlgEpFWi1
2,0000750929315523353,organic_search,2017-02-03 21:35:41,1.0,f6X79WOxA3jUxPU
3,0000750929315523353,dead_end,2017-02-03 21:35:42,0.0,f6X79WOxA3jUxPU
4,000094154452308463,display,2017-09-17 01:24:48,2193.0,L5wAoLtd1C6Cvvc
5,000094154452308463,display,2017-09-17 02:01:20,1.0,L5wAoLtd1C6Cvvc
6,000094154452308463,dead_end,2017-09-17 02:01:21,0.0,L5wAoLtd1C6Cvvc
7,000106364568517340,direct,2017-10-12 01:02:05,98.0,maGY5aACPGJoF9o
8,000106364568517340,dead_end,2017-10-12 01:03:43,0.0,maGY5aACPGJoF9o
9,0001348344294054462,direct,2017-06-05 15:06:24,1.0,VCl4OtCpaCafWUv


In [55]:
transactions_final_df = pd.concat([transactions_df,transactions_df2],ignore_index=True)
transactions_final_df = transactions_final_df[['sequence_id','fullVisitorId','event_name','event_datetime','conversion_proximity']]
transactions_final_df.sort_values(by=['sequence_id','conversion_proximity'],ascending=[True,False],inplace=True)

In [56]:
transactions_final_df.to_csv('../datasets/sequence_fact.csv',index=False)

In [57]:
visitor_detail_df.to_csv('../datasets/visitor_detail.csv', index=False)